<a href="https://colab.research.google.com/github/Tarikul1994/WeatherWise/blob/Tarikul1994-patch-1/starter_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# WeatherWise: Intelligent Weather Analysis & Advisory System

## Setup and Configuration

This section imports required libraries. Make sure to install `requests`, `pyinputplus`, and `matplotlib` if they are not available.


In [ ]:
import requests
import re
import datetime as dt
import matplotlib.pyplot as plt
import pyinputplus as pyip


## Weather Data Functions

Define the function to retrieve weather data using the wttr.in service. The function returns structured data including current conditions and forecast.

In [ ]:
def get_weather_data(location: str, forecast_days: int = 5) -> dict:
    """
    Retrieve weather data for a specified location using wttr.in (JSON).
    Uses 'requests' per assignment guidance. Handles incomplete data gracefully.
    """
    assert isinstance(location, str) and location.strip(), "location required"
    forecast_days = max(1, min(int(forecast_days), 5))
    url = f"https://wttr.in/{location}?format=j1"
    try:
        r = requests.get(url, timeout=15)
        r.raise_for_status()
        data = r.json()
    except Exception as e:
        return {"ok": False, "error": f"Network or parse error: {e}"}

    # Normalize shape: current + next N days (wttr gives 3 days; we cap to available)
    current = (data.get("current_condition") or [{}])[0]
    weather = data.get("weather") or []
    days = weather[:forecast_days]

    return {
        "ok": True,
        "provider": "wttr.in",
        "resolved_area": ((data.get("nearest_area") or [{}])[0].get("areaName") or [{"value": location}])[0]["value"],
        "current": {
            "tempC": float(current.get("temp_C") or 0),
            "windspeedKmph": float(current.get("windspeedKmph") or 0),
            "humidity": float(current.get("humidity") or 0),
            "weatherDesc": ((current.get("weatherDesc") or [{"value": ""}])[0]["value"]),
        },
        "days": [
            {
                "date": d.get("date"),
                "maxtempC": float(d.get("maxtempC") or 0),
                "mintempC": float(d.get("mintempC") or 0),
                "hourly": d.get("hourly") or [],
                "precipProb": max([int(h.get("chanceofrain") or 0) for h in (d.get("hourly") or [])] or [0]),
            } for d in days
        ],
    }


## Natural Language Processing Functions

Parse a plain-English weather question to extract the desired attribute, time window, and location.

In [ ]:
def parse_weather_question(question: str) -> dict:
    """
    Extract location, time window, and weather attribute from a plain-English question.
    Lightweight rules + regex so it's testable and transparent.
    """
    q = (question or "").strip().lower()

    # Attribute mapping
    attr_map = {
        "rain": ["rain", "umbrella", "precip", "shower", "wet"],
        "temp": ["temp", "temperature", "hot", "cold", "warm", "cool", "heat"],
        "wind": ["wind", "gust", "breeze"],
    }
    attribute = "temp"
    for k, words in attr_map.items():
        if any(w in q for w in words):
            attribute = k
            break

    # Time (basic)
    days_ahead = 0
    if "today" in q:
        days_ahead = 0
    elif "tomorrow" in q:
        days_ahead = 1
    else:
        m = re.search(r"in\s+(\d+)\s+days?", q)
        if m:
            days_ahead = int(m.group(1))

    # Location (simple heuristic: last 'in X')
    loc = None
    m = re.search(r"in\s+([a-zA-Z\s\-]+)$", question.strip())
    if m:
        loc = m.group(1).strip().rstrip("?.")

    return {
        "attribute": attribute,
        "days_ahead": days_ahead,
        "location": loc,
    }


## Response Generation

Generate a natural language response based on the parsed question and fetched weather data.

In [ ]:
def generate_weather_response(parsed_question: dict, weather_data: dict) -> str:
    """
    Craft a natural-language answer based on parsed intent and fetched data.
    Handles missing pieces with gentle prompts.
    """
    if not weather_data.get("ok"):
        return f"Sorry—couldn't fetch weather: {weather_data.get('error','unknown error')} ."

    city = weather_data.get("resolved_area") or "that location"
    days = weather_data.get("days") or []
    if not days:
        return f"I couldn’t find a forecast for {city}."

    idx = min(parsed_question.get("days_ahead", 0), len(days)-1)
    d = days[idx]
    attr = parsed_question.get("attribute", "temp")

    if attr == "rain":
        prob = d.get("precipProb", 0)
        hint = "Take an umbrella." if prob >= 50 else "Low chance of showers."
        return f"In {city} on {d['date']}, rain chance is about {prob}%. {hint}"
    elif attr == "wind":
        hourly = d.get("hourly", [])
        max_wind = max([float(h.get("WindspeedKmph") or 0) for h in hourly] or [0])
        return f"In {city} on {d['date']}, peak wind is around {round(max_wind)} km/h."
    else:  # temp
        return (f"In {city} on {d['date']}, expect ~{round(d['maxtempC'])}°C max "
                f"and ~{round(d['mintempC'])}°C min.")


## Visualisation Functions

Two functions to visualise temperature and precipitation over the forecast period.

In [ ]:
def create_temperature_visualisation(weather_data: dict, output_type: str = 'display'):
    """
    Line chart of min/max temps across the available days.
    """
    days = weather_data.get("days") or []
    if not days:
        raise ValueError("No day data to plot.")

    x = [d["date"] for d in days]
    tmax = [d["maxtempC"] for d in days]
    tmin = [d["mintempC"] for d in days]

    fig, ax = plt.subplots(figsize=(7,4))
    ax.plot(x, tmax, marker="o", label="Max °C")
    ax.plot(x, tmin, marker="o", label="Min °C")
    ax.set_xlabel("Day"); ax.set_ylabel("Temperature (°C)")
    ax.set_title("Temperature Outlook")
    ax.legend(); ax.grid(True, alpha=0.3)
    plt.xticks(rotation=30, ha="right")
    if output_type == 'figure':
        return fig
    plt.show()


In [ ]:
def create_precipitation_visualisation(weather_data: dict, output_type: str = 'display'):
    """
    Bar chart of daily rain probability (max across the day).
    """
    days = weather_data.get("days") or []
    if not days:
        raise ValueError("No day data to plot.")

    x = [d["date"] for d in days]
    p = [d.get("precipProb", 0) for d in days]

    fig, ax = plt.subplots(figsize=(7,4))
    ax.bar(x, p)
    ax.set_xlabel("Day"); ax.set_ylabel("Rain Probability (%)")
    ax.set_ylim(0, 100); ax.set_title("Rain Chances")
    plt.xticks(rotation=30, ha="right")
    if output_type == 'figure':
        return fig
    plt.show()


## User Interface

Use a simple menu to interact with the user via the console. The menu calls the functions defined above. Note: for notebook environments, user input may not work; run in a terminal for full functionality.

In [ ]:
def main_menu():
    while True:
        choice = pyip.menu(['Ask a question', 'Show charts', 'Quit'], numbered=True)
        if choice == 'Ask a question':
            q = pyip.inputStr("Your weather question: ")
            parsed = parse_weather_question(q)
            loc = parsed['location'] or pyip.inputStr("Location (e.g., Perth): ")
            data = get_weather_data(loc, forecast_days=5)
            print(generate_weather_response(parsed | {'location': loc}, data))
        elif choice == 'Show charts':
            loc = pyip.inputStr("Location: ")
            data = get_weather_data(loc, forecast_days=5)
            if not data.get("ok"):
                print(data.get("error"))
                continue
            create_temperature_visualisation(data)
            create_precipitation_visualisation(data)
        else:
            break

# Uncomment the following line to run the menu in a standalone environment
# main_menu()


## Testing and Examples

The following cells show example usage of the functions defined above. Adjust the location and number of forecast days as desired.

In [ ]:
# Example usage:
# Retrieve weather data for Perth with a 3-day forecast
# data = get_weather_data('Perth', forecast_days=3)
# print(data)

# Generate a response for a question
# parsed = parse_weather_question('Will it rain tomorrow in Sydney?')
# print(generate_weather_response(parsed, get_weather_data(parsed['location'] or 'Sydney', forecast_days=2)))

# Plot visualisations
# create_temperature_visualisation(data)
# create_precipitation_visualisation(data)
